In [11]:

#old model

In [12]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier
from datetime import date

def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_ema(series, span):
    return series.ewm(span=span, adjust=False).mean()

def calculate_macd(series, fast=12, slow=26, signal=9):
    ema_fast = calculate_ema(series, fast)
    ema_slow = calculate_ema(series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = calculate_ema(macd_line, signal)
    return macd_line, signal_line

def predict_stock_direction(ticker):
    try:
        print(f" Downloading data for: {ticker}")
        data = yf.download(ticker, start="2020-01-01", end=date.today(), progress=False, auto_adjust=False)

        if data.empty:
            print(" No data returned. Please check the ticker symbol.")
            return

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = ['_'.join(col).lower() for col in data.columns.values]
        else:
            data.columns = data.columns.str.lower()

        expected = {
            f'open_{ticker.lower()}': 'open',
            f'high_{ticker.lower()}': 'high',
            f'low_{ticker.lower()}': 'low',
            f'close_{ticker.lower()}': 'close',
            f'volume_{ticker.lower()}': 'volume',
        }
        data.rename(columns=expected, inplace=True)

        required_cols = ['open', 'high', 'low', 'close', 'volume']
        if not all(col in data.columns for col in required_cols):
            return

        data['rsi'] = calculate_rsi(data['close'], period=14)
        macd_line, signal_line = calculate_macd(data['close'], fast=12, slow=26, signal=9)
        data['macd'] = macd_line
        data['ema_50'] = calculate_ema(data['close'], span=50)
        data['ema_200'] = calculate_ema(data['close'], span=200)
        data['ema_diff'] = data['ema_50'] - data['ema_200']
        data['lag1_return'] = data['close'].pct_change().shift(1)
        data['target'] = (data['close'].shift(-1) > data['close']).astype(int)
        data.dropna(inplace=True)

        features = ['open', 'high', 'low', 'close', 'volume', 'rsi', 'macd', 'ema_diff', 'lag1_return']
        X = data[features]
        y = data['target']

        model = RandomForestClassifier(n_estimators=100, min_samples_split=50, random_state=42)
        model.fit(X, y)

        latest = X.iloc[-1].values.reshape(1, -1)
        prediction = model.predict(latest)[0]
        confidence = model.predict_proba(latest)[0][prediction]

        direction = "UP" if prediction == 1 else "DOWN"

        print("\n" + "="*60)
        print(f" PREDICTION FOR {ticker.upper()} - NEXT TRADING DAY")
        print(f" The price is predicted to go: {direction}")
        print(f" Confidence: {confidence:.2%}")
        print("="*60)

    except Exception as e:
        print(f" ERROR: {e}")

if __name__ == "__main__":
    predict_stock_direction("AAPL") #change the stock here



 PREDICTION FOR AAPL - NEXT TRADING DAY
 The price is predicted to go: DOWN
 Confidence: 53.90%


/home/subhash/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/subhash/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
